In [1]:
import sys

sys.path.append('./Belopsem/code/') # TO ADAPT

In [2]:
# Importing Belopsem
import Belopsem.code.utils as utils
import Belopsem.code.bucc_style_dataset as bsd


In [3]:
import random
import unicodedata

In [4]:
cd PATH_TO_WORKSPACE

/dss/dssmcmlfs01/pn39je/pn39je-dss-0001/go25puh


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
# Additional functions to pre-process the original data
# TO MODIFY if needed
def remove_head_id(split_dataset):
    '''Remove original sentence ID in the news dataset (Leipzig corpus).'''
    new_split_dataset = [sentence.split('\t')[1] for sentence in split_dataset] 
    return new_split_dataset

def basic_processing(data_file):
    '''Applying Unicode normalisation and spliting sentences.'''
    normalised_text = unicodedata.normalize('NFC', data_file)
    split_text = utils.text_to_line(normalised_text)
    print(len(split_text), len(set(split_text)))
    return split_text

def filter_sentence_3(split_dataset):
    '''Filter out sentences with less than X or more than X words.'''
    print(f'Before: {len(split_dataset)}')
    filtered_dataset = [sentence.strip() for sentence in split_dataset 
                        if (len(sentence.split(' ')) < 40)] #(len(sentence.split(' ')) > 5 and 
    print(f'After: {len(filtered_dataset)}')
    return filtered_dataset

In [6]:
random.seed(42)

## Create a dataset for the Occitan-Spanish pair

In [7]:
# Import texts
monolingual_oci = open('raw_data/raw_oci-es/oci_wikipedia_2021_30K-sentences.txt', 'r').read() # monolingual source
monolingual_es = open('raw_data/raw_oci-es/spa_wikipedia_2021_30K-sentences.txt', 'r').read() # monolingual target
parallel_oci = open('raw_data/raw_oci-es/wikimedia.es-oc.oc', 'r').read() # parallel source
parallel_es = open('raw_data/raw_oci-es/wikimedia.es-oc.es', 'r').read() # parallel target


In [8]:
# Preprocessing
split_monolingual_oci = basic_processing(monolingual_oci)
split_monolingual_es = basic_processing(monolingual_es)
#30000
split_parallel_oci = basic_processing(parallel_oci)
split_parallel_es = basic_processing(parallel_es)
#1980

split_monolingual_oci[0:5], split_parallel_es[0:5]

30000 30000
30000 30000
1980 1941
1980 1935


(["1\t²per sos trabalhs sus las reaccions d'",
  "2\t» (46 Estats membres, contra 27 per l'",
  '3\tA 10 km al sud, se tr ba Cap de lu c de Costat : Rebais.',
  "4\tA 1.326 m tres d'altitud, es la vila mai nauta de França.",
  '5\tA 14 oras, lo desplegament de la polícia compren un elicoptèr e de desena de veïculs (polícia, pompièrs, secors).'],
 ['Obras ',
  'Enlaces externos ',
  'Josiana Ubaud, en francés, Josiane Ubaud (Marsella, Francia, 10 de mayo de 1947) es una lingüista especializada en lexicografía y gramaticografía y una botánica occitana. ',
  'Es la autora de dos obras esenciales para el conocimiento del occitano moderno, el Diccionario ortográfico, gramatical y morfológico del occitano y el Diccionario científico francés-occitano.[1] ',
  'Josiana Ubaud fue profesora de matemáticas durante ocho años antes de advocar a la lexicografía occitana después de regresar al país occitano en 1990. '])

In [9]:
# Filtering monolingual data
# Sentences from Leipzig corpus should have their ID removed
filter_split_mono_oci = remove_head_id(filter_sentence_3(split_monolingual_oci))
filter_split_mono_es = remove_head_id(filter_sentence_3(split_monolingual_es))
print(filter_split_mono_oci[0:5], filter_split_mono_es[0:5])

Before: 30000
After: 29674
Before: 30000
After: 29177
["²per sos trabalhs sus las reaccions d'", "» (46 Estats membres, contra 27 per l'", 'A 10 km al sud, se tr ba Cap de lu c de Costat : Rebais.', "A 1.326 m tres d'altitud, es la vila mai nauta de França.", 'A 14 oras, lo desplegament de la polícia compren un elicoptèr e de desena de veïculs (polícia, pompièrs, secors).'] ['; 12 de abril de 1996) es una actriz de la industria del entretenimiento para adultos y una personalidad de internet.', '"15 Y el Señor dijo a Noé: La hijas de tus hijos se han vendido; por lo que, he aquí, mi ira está encendida en contra de los hijos de los hombres, porque no quieren escuchar mi voz".', '” (1948 - 1958) los grupos paramilitares del conservatismo inician una persecución contra los campesinos liberales y comunistas.', '; 1954), conocido simplemente como Vacca es un artista español.', '" (1957) después de que el temperamental Freda también hubiera salido del set de esa película después de sólo unos 

In [10]:
# Create the training and test synthetic corpora
oci_es_corpora = bsd.split_shuffle_create_corpus(
     filter_split_mono_oci, filter_split_mono_es, split_parallel_oci, split_parallel_es)

Dataset length: 29674.
7418 22256
Dataset length: 29177.
7294 21883
Dataset length: 1980.
495 1485
Dataset length: 1980.
495 1485
False False False False
7418 monolingual source sentences.
7294 monolingual target sentences.
Whole corpus:
7899 monolingual source sentences.
7780 monolingual target sentences.
src-0000000	Aquela annada ( trg-0000000	Los cabildos abiertos atribuían a la parte representativa de la ciudad el derecho a deliberar sobre cuestiones que por su naturaleza requerían una solución extraordinaria.
22256 monolingual source sentences.
21883 monolingual target sentences.
Whole corpus:
23675 monolingual source sentences.
23334 monolingual target sentences.
src-0000000	E que mei, ua cavitat que serveish tà formar espeleològs. trg-0000000	Los ingresos medios en las Tierras APY eran significativamente más bajos que en el resto de Australia, con un ingreso medio semanal de $219/semana, comparado con los $466/semana a nivel nacional.


In [11]:
# Saving the files
src = 'oci'
trg = 'es'
main_path = f'./data/{src}-{trg}' # CHANGE SAVE PATH

bsd.save_files(src, trg, main_path, oci_es_corpora)

In [12]:
!wc -l ./data/bucc2017/oci-es/oci-es.*

  23333 UnsupPSE/data/bucc2017/oci-es/oci-es.test.es
   1456 UnsupPSE/data/bucc2017/oci-es/oci-es.test.gold
  23674 UnsupPSE/data/bucc2017/oci-es/oci-es.test.oci
   7779 UnsupPSE/data/bucc2017/oci-es/oci-es.train.es
    485 UnsupPSE/data/bucc2017/oci-es/oci-es.train.gold
   7898 UnsupPSE/data/bucc2017/oci-es/oci-es.train.oci
  64625 total
